<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
Inf
</b></h1> 

### **[FE](Melspec PCEN melspec**1.5)**

<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
Exp Info
</b></h1> 

```
D-300-TRAIN-EF_MELSPEC_PCEN-tf_efficientnet_b0_ns-SizeExp4-f0..4
IMAGE_RESIZE_RATE=2.0


0.731797545 ---low fold0
0.737641303
0.73400335
0.731386122 ---low fold4
0.73463218

OOF:0.7338921
```

```
ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f0
0.742723319  ---low fold0
0.75465313
0.75177305
0.745695364  ---low fold3
0.764919222
0.747076512  ---low fold5
0.754109359
0.753623188
0.738317757  ---low fold8
0.759451322
OOF:0.751234222
```

```
ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f0
0.754087421
0.755778894
0.75276012   ---low fold2
0.761329305
0.75862069
0.761050183
0.769940672
0.759133965
0.750921891  ---low fold8
0.752230611  ---low fold9
OOF:0.757585375
```

```
ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f0
0.738659445  ---low fold0
0.752422319
0.748        ---low fold2
0.732488326  ---low fold3
0.755449591
0.750830565
0.755584757
0.74775673
0.741903172  ---low fold8
0.75889859
OOF:0.748199349
```

<a id=#cbb></a>
<h2 style="color: #6cb4e4; background: #dfefff;  box-shadow: 0px 0px 0px 5px #dfefff;  border: dashed 4px white;  padding: 0.2em 0.5em;">
<b>
External Lib
</b></h2> 

In [ ]:
!pip install ../input/torchlibrosa/torchlibrosa-0.0.5-py3-none-any.whl > /dev/null

In [ ]:
import cv2
import audioread
import logging
import os
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import random
import time
import warnings

import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata

from contextlib import contextmanager
from pathlib import Path
from typing import List
from typing import Optional
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, GroupKFold

from albumentations.core.transforms_interface import ImageOnlyTransform
from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation
from tqdm import tqdm

import albumentations as A
import albumentations.pytorch.transforms as T
import matplotlib.pyplot as plt

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
    
def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger
    
    
@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)


def get_device() -> torch.device:
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


device = get_device()
logger = get_logger("main.log")
set_seed(42)

In [ ]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        model.weight.data.normal_(0, 0.01)
        model.bias.data.zero_()


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    output = F.interpolate(
        framewise_output.unsqueeze(1),
        size=(frames_num, framewise_output.size(2)),
        align_corners=True,
        mode="bilinear").squeeze(1)

    return output



class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


class TimmSED(nn.Module):
    def __init__(self, base_model_name: str, pretrained=False, num_classes=24, in_channels=1):
        super().__init__()

        self.spec_augmenter = SpecAugmentation(time_drop_width=64//2, time_stripes_num=2,
                                               freq_drop_width=8//2, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(int(CFG.n_mels*CFG.IMAGE_RESIZE_RATE))

        base_model = timm.create_model(
            base_model_name, pretrained=pretrained, in_chans=in_channels)
        layers = list(base_model.children())[:-2]
        self.encoder = nn.Sequential(*layers)

        if hasattr(base_model, "fc"):
            in_features = base_model.fc.in_features
        else:
            in_features = base_model.classifier.in_features

        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlockV2(
            in_features, num_classes, activation="sigmoid")

        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        

    def forward(self, input_data):
        x = input_data # (batch_size, 3, time_steps, mel_bins)

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            if random.random() < 0.25:
                x = self.spec_augmenter(x)

        x = x.transpose(2, 3)

        x = self.encoder(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        interpolate_ratio = frames_num // segmentwise_output.size(1)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            'framewise_output': framewise_output,
            'clipwise_output': clipwise_output,
            'logit': logit,
            'framewise_logit': framewise_logit,
        }

        return output_dict

<a id=#cbb></a>
<h2 style="color: #6cb4e4; background: #dfefff;  box-shadow: 0px 0px 0px 5px #dfefff;  border: dashed 4px white;  padding: 0.2em 0.5em;">
<b>
TTA
</b></h2> 

In [ ]:
mean = (0.485, 0.456, 0.406) # RGB
std = (0.229, 0.224, 0.225) # RGB

albu_transforms = {
    'train' : A.Compose([
            A.HorizontalFlip(p=0.5),
            A.OneOf([
                A.Cutout(max_h_size=5, max_w_size=16),
                A.CoarseDropout(max_holes=4),
            ], p=0.5),
            A.Normalize(mean, std),
    ]),
    'valid' : A.Compose([
            A.Normalize(mean, std),
    ]),
}

<a id=#cbb></a>
<h2 style="color: #6cb4e4; background: #dfefff;  box-shadow: 0px 0px 0px 5px #dfefff;  border: dashed 4px white;  padding: 0.2em 0.5em;">
<b>
CFG
</b></h2> 

In [ ]:
class CFG:
    IMAGE_RESIZE_RATE=2.0
    EXP_ID = '018' 

    ######################
    # Globals #
    ######################
    seed = 42
    epochs = 5
    train = True
    folds = [0, 1, 2, 3, 4]
    img_size = 224
    main_metric = "epoch_f1_at_03"
    minimize_metric = False

    ######################
    # Dataset #
    ######################
    transforms = {
        "train": [{"name": "Normalize"}],
        "valid": [{"name": "Normalize"}]
    }
    period = 5
    n_mels = 224
    fmin = 20
    fmax = 16000
    n_fft = 2048
    hop_length = 512
    sample_rate = 32000
    melspectrogram_parameters = {
        "n_mels": 224,
        "fmin": 20,
        "fmax": 16000
    }

    target_columns = 'afrsil1 akekee akepa1 akiapo akikik amewig aniani apapan arcter \
                      barpet bcnher belkin1 bkbplo bknsti bkwpet blkfra blknod bongul \
                      brant brnboo brnnod brnowl brtcur bubsan buffle bulpet burpar buwtea \
                      cacgoo1 calqua cangoo canvas caster1 categr chbsan chemun chukar cintea \
                      comgal1 commyn compea comsan comwax coopet crehon dunlin elepai ercfra eurwig \
                      fragul gadwal gamqua glwgul gnwtea golphe grbher3 grefri gresca gryfra gwfgoo \
                      hawama hawcoo hawcre hawgoo hawhaw hawpet1 hoomer houfin houspa hudgod iiwi incter1 \
                      jabwar japqua kalphe kauama laugul layalb lcspet leasan leater1 lessca lesyel lobdow lotjae \
                      madpet magpet1 mallar3 masboo mauala maupar merlin mitpar moudov norcar norhar2 normoc norpin \
                      norsho nutman oahama omao osprey pagplo palila parjae pecsan peflov perfal pibgre pomjae puaioh \
                      reccar redava redjun redpha1 refboo rempar rettro ribgul rinduc rinphe rocpig rorpar rudtur ruff \
                      saffin sander semplo sheowl shtsan skylar snogoo sooshe sooter1 sopsku1 sora spodov sposan \
                      towsol wantat1 warwhe1 wesmea wessan wetshe whfibi whiter whttro wiltur yebcar yefcan zebdov'.split()

    ######################
    # Loaders #
    ######################
    loader_params = {
        "train": {
            "batch_size": 32, 
            "num_workers": 0,
            "shuffle": True
        },
        "valid": {
            "batch_size": 64,
            "num_workers": 0,
            "shuffle": False
        }
    }

    ######################
    # Model #
    ######################
#     base_model_name = "tf_efficientnet_b0_ns"
    pooling = "max"
    pretrained = False
    num_classes = 152
    in_channels = 3

    N_FOLDS = 5
    LR = 1e-3
    T_max=10
    min_lr=1e-6

In [ ]:
AUDIO_PATH = '../input/birdclef-2022/train_audio'
CLASSES = sorted(os.listdir(AUDIO_PATH))
NUM_CLASSES = len(CLASSES)
class AudioParams:
    """
    Parameters used for the audio data
    """
    sr = 32000
    duration = 5
    # Melspectrogram
    n_mels = 224
    fmin = 20
    fmax = 16000

In [ ]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V


"""
    Feature Engineering
    * ch1:Melspec
    * ch2:PCEN
    * ch3:Melspec ** 1.5
"""
def compute_melspec_pcen(y, params):
    melspec = librosa.feature.melspectrogram(
        y=y, sr=params.sr, n_mels=params.n_mels, fmin=params.fmin, fmax=params.fmax,
    )
    
    pcen = librosa.pcen(melspec,
                        sr=CFG.sample_rate,
                        time_constant=0.06,
                        eps=1e-6,
                        gain=0.8,
                        power=0.25,
                        bias=10,
                        hop_length=CFG.hop_length
                       )
    clean_mel = librosa.power_to_db(melspec ** 1.5)
    
    # Feature Normalize Ch3
    melspec = librosa.power_to_db(melspec)
    norm_melspec = normalize_melspec(melspec)
    norm_pcen = normalize_melspec(pcen)
    norm_clean_mel = normalize_melspec(clean_mel)
    
    # Convert Image
    image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)
    height, width, _ = image.shape
    image = cv2.resize(image, 
                   (int(int(width * 224 / height)*CFG.IMAGE_RESIZE_RATE), int(224*CFG.IMAGE_RESIZE_RATE))
                  )
    return image

In [ ]:
def compute_melspec(y, params):
    """
    Computes a mel-spectrogram and puts it at decibel scale
    Arguments:
        y {np array} -- signal
        params {AudioParams} -- Parameters to use for the spectrogram. Expected to have the attributes sr, n_mels, f_min, f_max
    Returns:
        np array -- Mel-spectrogram
    """
    melspec = librosa.feature.melspectrogram(
        y=y, sr=params.sr, n_mels=params.n_mels, fmin=params.fmin, fmax=params.fmax,
    )

    melspec = librosa.power_to_db(melspec).astype(np.float32)
    return melspec


def mono_to_color(X, eps=1e-6, mean=None, std=None):
    """
    Converts a one channel array to a 3 channel one in [0, 255]
    Arguments:
        X {numpy array [H x W]} -- 2D array to convert
    Keyword Arguments:
        eps {float} -- To avoid dividing by 0 (default: {1e-6})
        mean {None or np array} -- Mean for normalization (default: {None})
        std {None or np array} -- Std for normalization (default: {None})
    Returns:
        numpy array [3 x H x W] -- RGB numpy array
    """
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)

    # Normalize to [0, 255]
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

In [ ]:
class TestDataset(torchdata.Dataset):
    def __init__(self, df: pd.DataFrame, clip: np.ndarray):
        self.df = df
        # self.clip = clip
        self.clip = np.concatenate([clip, clip, clip])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        SR = 32000
        sample = self.df.loc[idx, :]
        row_id = sample.row_id

        end_seconds = int(sample.seconds)
        start_seconds = int(end_seconds - 5)
        
        # end_index = int(SR * (end_seconds + (self.train_period - 5) / 2) + len(self.clip) // 3)
        # start_index = int(SR * (start_seconds - (self.train_period - 5) / 2) + len(self.clip) // 3)
        
        # y = self.clip[start_index:end_index].astype(np.float32)
        image = self.clip[SR*start_seconds:SR*end_seconds].astype(np.float32)
        image = np.nan_to_num(image)
        
#         image = compute_melspec(image, AudioParams)
#         image = mono_to_color(image)
#         image = image.astype(np.uint8)
        image = compute_melspec_pcen(image, AudioParams)
        image = image.astype(np.float32)


        image = albu_transforms['valid'](image=image)['image'].T
            
        return {
            "image": image,
            "row_id": row_id,
        }

<a id=#cbb></a>
<h2 style="color: #6cb4e4; background: #dfefff;  box-shadow: 0px 0px 0px 5px #dfefff;  border: dashed 4px white;  padding: 0.2em 0.5em;">
<b>
Inf
</b></h2> 

In [ ]:
model_paths = [
               '../input/birdclef2022-dc-d-300-20220504004758/D-300-TRAIN-EF_MELSPEC_PCEN-tf_efficientnet_b0_ns-SizeExp4-f0/fold-0.bin',
#                '../input/birdclef2022-dc-d-300-20220504004758/D-300-TRAIN-EF_MELSPEC_PCEN-tf_efficientnet_b0_ns-SizeExp4-f1/fold-1.bin',
#                '../input/birdclef2022-dc-d-300-20220504004758/D-300-TRAIN-EF_MELSPEC_PCEN-tf_efficientnet_b0_ns-SizeExp4-f2/fold-2.bin',
#                '../input/birdclef2022-dc-d-300-20220504004758/D-300-TRAIN-EF_MELSPEC_PCEN-tf_efficientnet_b0_ns-SizeExp4-f3/fold-3.bin',
               '../input/birdclef2022-dc-d-300-20220504004758/D-300-TRAIN-EF_MELSPEC_PCEN-tf_efficientnet_b0_ns-SizeExp4-f4/fold-4.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f0/fold-0.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f1/fold-1.bin',
               '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f2/fold-2.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f3/fold-3.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f4/fold-4.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f5/fold-5.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f6/fold-6.bin',
#                '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f7/fold-7.bin',
               '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f8/fold-8.bin',
               '../input/birdclef2022-dc-za-500-20220524030848/ZA-500-TRAIN-PCEN-tf_efficientnetv2_l_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f9/fold-9.bin',
               '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f0/fold-0.bin',
#                '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f1/fold-1.bin',
#                '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f2/fold-2.bin',
               '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f3/fold-3.bin',
#                '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f4/fold-4.bin',
               '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f5/fold-5.bin',
#                '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f6/fold-6.bin',
#                '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f7/fold-7.bin',
               '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f8/fold-8.bin',
#                '../input/birdclef2022-dc-za-320-20220518154439/ZA-320-TRAIN-PCEN-tf_efficientnetv2_s_in21ft1k-Split10-Resizex20-BGM-DaExp0-AllFold-f9/fold-9.bin',
               '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f0/fold-0.bin',
#                '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f1/fold-1.bin',
               '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f2/fold-2.bin',
               '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f3/fold-3.bin',
#                '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f4/fold-4.bin',
#                '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f5/fold-5.bin',
#                '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f6/fold-6.bin',
#                '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f7/fold-7.bin',
               '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f8/fold-8.bin',
#                '../input/birdclef2022-dc-za-300-20220517141400/ZA-300-TRAIN-PCEN-tf_efficientnet_b0_ns-Split10-Resizex20-BGM-DaExp0-AllFold-f9/fold-9.bin',
]
model_arc = [
             'tf_efficientnet_b0_ns',
             'tf_efficientnet_b0_ns',
#              'tf_efficientnetv2_l_in21ft1k',
#              'tf_efficientnetv2_l_in21ft1k',
#              'tf_efficientnetv2_l_in21ft1k',
#              'tf_efficientnetv2_l_in21ft1k',
#              'tf_efficientnetv2_l_in21ft1k',
#              'tf_efficientnetv2_l_in21ft1k',
#              'tf_efficientnetv2_l_in21ft1k',
             'tf_efficientnetv2_l_in21ft1k',
             'tf_efficientnetv2_l_in21ft1k',
             'tf_efficientnetv2_l_in21ft1k',
             'tf_efficientnetv2_s_in21ft1k',
             'tf_efficientnetv2_s_in21ft1k',
             'tf_efficientnetv2_s_in21ft1k',
             'tf_efficientnetv2_s_in21ft1k',
#              'tf_efficientnetv2_s_in21ft1k',
#              'tf_efficientnetv2_s_in21ft1k',
#              'tf_efficientnetv2_s_in21ft1k',
#              'tf_efficientnetv2_s_in21ft1k',
#              'tf_efficientnetv2_s_in21ft1k',
#              'tf_efficientnetv2_s_in21ft1k',
             'tf_efficientnet_b0_ns',
             'tf_efficientnet_b0_ns',
             'tf_efficientnet_b0_ns',
             'tf_efficientnet_b0_ns',
#              'tf_efficientnet_b0_ns',
#              'tf_efficientnet_b0_ns',
#              'tf_efficientnet_b0_ns',
#              'tf_efficientnet_b0_ns',
#              'tf_efficientnet_b0_ns',
#              'tf_efficientnet_b0_ns',
]

In [ ]:
models = []
for _model_arc , p in zip(model_arc, model_paths):
    model = TimmSED(
        base_model_name=_model_arc,
        pretrained=CFG.pretrained,
        num_classes=CFG.num_classes,
        in_channels=CFG.in_channels)
    
    model.to(device)
    model.load_state_dict(torch.load(p,map_location='cuda:0'))
    model.eval()
    models.append(model)
    
print()

In [ ]:
TARGET_SR = 32000
DATADIR = Path("../input/birdclef-2022/test_soundscapes/")

In [ ]:
all_audios = list(DATADIR.glob("*.ogg"))
sample_submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')
sample_submission


In [ ]:
IS_SCORED_BIRD_W = 1.0
ISNOT_SCORED_BIRD_W = 1.0

BIRD_WEIGHT=[
    ISNOT_SCORED_BIRD_W, #'afrsil1'	0
    ISNOT_SCORED_BIRD_W, #'akekee'	1
    ISNOT_SCORED_BIRD_W, #'akepa1'	2
    IS_SCORED_BIRD_W, #'akiapo'	3  ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'akikik'	4
    ISNOT_SCORED_BIRD_W, #'amewig'	5
    IS_SCORED_BIRD_W, #'aniani'	6  ★Scored_bird
    IS_SCORED_BIRD_W, #'apapan'	7  ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'arcter'	8
    IS_SCORED_BIRD_W, #'barpet'	9  ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'bcnher'	10
    ISNOT_SCORED_BIRD_W, #'belkin1'	11
    ISNOT_SCORED_BIRD_W, #'bkbplo'	12
    ISNOT_SCORED_BIRD_W, #'bknsti'	13
    ISNOT_SCORED_BIRD_W, #'bkwpet'	14
    ISNOT_SCORED_BIRD_W, #'blkfra'	15
    ISNOT_SCORED_BIRD_W, #'blknod'	16
    ISNOT_SCORED_BIRD_W, #'bongul'	17
    ISNOT_SCORED_BIRD_W, #'brant'	18
    ISNOT_SCORED_BIRD_W, #'brnboo'	19
    ISNOT_SCORED_BIRD_W, #'brnnod'	20
    ISNOT_SCORED_BIRD_W, #'brnowl'	21
    ISNOT_SCORED_BIRD_W, #'brtcur'	22
    ISNOT_SCORED_BIRD_W, #'bubsan'	23
    ISNOT_SCORED_BIRD_W, #'buffle'	24
    ISNOT_SCORED_BIRD_W, #'bulpet'	25
    ISNOT_SCORED_BIRD_W, #'burpar'	26
    ISNOT_SCORED_BIRD_W, #'buwtea'	27
    ISNOT_SCORED_BIRD_W, #'cacgoo1'	28
    ISNOT_SCORED_BIRD_W, #'calqua'	29
    ISNOT_SCORED_BIRD_W, #'cangoo'	30
    ISNOT_SCORED_BIRD_W, #'canvas'	31
    ISNOT_SCORED_BIRD_W, #'caster1'	32
    ISNOT_SCORED_BIRD_W, #'categr'	33
    ISNOT_SCORED_BIRD_W, #'chbsan'	34
    ISNOT_SCORED_BIRD_W, #'chemun'	35
    ISNOT_SCORED_BIRD_W, #'chukar'	36
    ISNOT_SCORED_BIRD_W, #'cintea'	37
    ISNOT_SCORED_BIRD_W, #'comgal1'	38
    ISNOT_SCORED_BIRD_W, #'commyn'	39
    ISNOT_SCORED_BIRD_W, #'compea'	40
    ISNOT_SCORED_BIRD_W, #'comsan'	41
    ISNOT_SCORED_BIRD_W, #'comwax'	42
    ISNOT_SCORED_BIRD_W, #'coopet'	43
    IS_SCORED_BIRD_W, #'crehon'	44 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'dunlin'	45
    IS_SCORED_BIRD_W, #'elepai'	46 ★Scored_bird
    IS_SCORED_BIRD_W, #'ercfra'	47 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'eurwig'	48
    ISNOT_SCORED_BIRD_W, #'fragul'	49
    ISNOT_SCORED_BIRD_W, #'gadwal'	50
    ISNOT_SCORED_BIRD_W, #'gamqua'	51
    ISNOT_SCORED_BIRD_W, #'glwgul'	52
    ISNOT_SCORED_BIRD_W, #'gnwtea'	53
    ISNOT_SCORED_BIRD_W, #'golphe'	54
    ISNOT_SCORED_BIRD_W, #'grbher3'	55
    ISNOT_SCORED_BIRD_W, #'grefri'	56
    ISNOT_SCORED_BIRD_W, #'gresca'	57
    ISNOT_SCORED_BIRD_W, #'gryfra'	58
    ISNOT_SCORED_BIRD_W, #'gwfgoo'	59
    IS_SCORED_BIRD_W, #'hawama'	60 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'hawcoo'	61
    IS_SCORED_BIRD_W, #'hawcre'	62 ★Scored_bird
    IS_SCORED_BIRD_W, #'hawgoo'	63 ★Scored_bird
    IS_SCORED_BIRD_W, #'hawhaw'	64 ★Scored_bird
    IS_SCORED_BIRD_W, #'hawpet1'	65 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'hoomer'	66
    IS_SCORED_BIRD_W, #'houfin'	67 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'houspa'	68
    ISNOT_SCORED_BIRD_W, #'hudgod'	69
    IS_SCORED_BIRD_W, #'iiwi'	70 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'incter1'	71
    IS_SCORED_BIRD_W, #'jabwar'	72 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'japqua'	73
    ISNOT_SCORED_BIRD_W, #'kalphe'	74
    ISNOT_SCORED_BIRD_W, #'kauama'	75
    ISNOT_SCORED_BIRD_W, #'laugul'	76
    ISNOT_SCORED_BIRD_W, #'layalb'	77
    ISNOT_SCORED_BIRD_W, #'lcspet'	78
    ISNOT_SCORED_BIRD_W, #'leasan'	79
    ISNOT_SCORED_BIRD_W, #'leater1'	80
    ISNOT_SCORED_BIRD_W, #'lessca'	81
    ISNOT_SCORED_BIRD_W, #'lesyel'	82
    ISNOT_SCORED_BIRD_W, #'lobdow'	83
    ISNOT_SCORED_BIRD_W, #'lotjae'	84
    ISNOT_SCORED_BIRD_W, #'madpet'	85
    ISNOT_SCORED_BIRD_W, #'magpet1'	86
    ISNOT_SCORED_BIRD_W, #'mallar3'	87
    ISNOT_SCORED_BIRD_W, #'masboo'	88
    ISNOT_SCORED_BIRD_W, #'mauala'	89
    IS_SCORED_BIRD_W, #'maupar'	90 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'merlin'	91
    ISNOT_SCORED_BIRD_W, #'mitpar'	92
    ISNOT_SCORED_BIRD_W, #'moudov'	93
    ISNOT_SCORED_BIRD_W, #'norcar'	94
    ISNOT_SCORED_BIRD_W, #'norhar2'	95
    ISNOT_SCORED_BIRD_W, #'normoc'	96
    ISNOT_SCORED_BIRD_W, #'norpin'	97
    ISNOT_SCORED_BIRD_W, #'norsho'	98
    ISNOT_SCORED_BIRD_W, #'nutman'	99
    ISNOT_SCORED_BIRD_W, #'oahama'	100
    IS_SCORED_BIRD_W, #'omao'	101 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'osprey'	102
    ISNOT_SCORED_BIRD_W, #'pagplo'	103
    ISNOT_SCORED_BIRD_W, #'palila'	104
    ISNOT_SCORED_BIRD_W, #'parjae'	105
    ISNOT_SCORED_BIRD_W, #'pecsan'	106
    ISNOT_SCORED_BIRD_W, #'peflov'	107
    ISNOT_SCORED_BIRD_W, #'perfal'	108
    ISNOT_SCORED_BIRD_W, #'pibgre'	109
    ISNOT_SCORED_BIRD_W, #'pomjae'	110
    IS_SCORED_BIRD_W, #'puaioh'	111 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'reccar'	112
    ISNOT_SCORED_BIRD_W, #'redava'	113
    ISNOT_SCORED_BIRD_W, #'redjun'	114
    ISNOT_SCORED_BIRD_W, #'redpha1'	115
    ISNOT_SCORED_BIRD_W, #'refboo'	116
    ISNOT_SCORED_BIRD_W, #'rempar'	117
    ISNOT_SCORED_BIRD_W, #'rettro'	118
    ISNOT_SCORED_BIRD_W, #'ribgul'	119
    ISNOT_SCORED_BIRD_W, #'rinduc'	120
    ISNOT_SCORED_BIRD_W, #'rinphe'	121
    ISNOT_SCORED_BIRD_W, #'rocpig'	122
    ISNOT_SCORED_BIRD_W, #'rorpar'	123
    ISNOT_SCORED_BIRD_W, #'rudtur'	124
    ISNOT_SCORED_BIRD_W, #'ruff'	125
    ISNOT_SCORED_BIRD_W, #'saffin'	126
    ISNOT_SCORED_BIRD_W, #'sander'	127
    ISNOT_SCORED_BIRD_W, #'semplo'	128
    ISNOT_SCORED_BIRD_W, #'sheowl'	129
    ISNOT_SCORED_BIRD_W, #'shtsan'	130
    IS_SCORED_BIRD_W, #'skylar'	131 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'snogoo'	132
    ISNOT_SCORED_BIRD_W, #'sooshe'	133
    ISNOT_SCORED_BIRD_W, #'sooter1'	134
    ISNOT_SCORED_BIRD_W, #'sopsku1'	135
    ISNOT_SCORED_BIRD_W, #'sora'	136
    ISNOT_SCORED_BIRD_W, #'spodov'	137
    ISNOT_SCORED_BIRD_W, #'sposan'	138
    ISNOT_SCORED_BIRD_W, #'towsol'	139
    ISNOT_SCORED_BIRD_W, #'wantat1'	140
    IS_SCORED_BIRD_W, #'warwhe1'	141 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'wesmea'	142
    ISNOT_SCORED_BIRD_W, #'wessan'	143
    ISNOT_SCORED_BIRD_W, #'wetshe'	144
    ISNOT_SCORED_BIRD_W, #'whfibi'	145
    ISNOT_SCORED_BIRD_W, #'whiter'	146
    ISNOT_SCORED_BIRD_W, #'whttro'	147
    ISNOT_SCORED_BIRD_W, #'wiltur'	148
    ISNOT_SCORED_BIRD_W, #'yebcar'	149
    IS_SCORED_BIRD_W, #'yefcan'	150 ★Scored_bird
    ISNOT_SCORED_BIRD_W, #'zebdov'	151
]

In [ ]:
len(BIRD_WEIGHT)

In [ ]:
INFERENCE_BS=32

In [ ]:
def prediction_for_clip(test_df: pd.DataFrame, 
                        clip: np.ndarray, 
                        models, 
                        threshold=0.05, 
                        threshold_long=None):

    """
        Init dataset
    """
    dataset = TestDataset(df=test_df,clip=clip,)
    loader = torchdata.DataLoader(dataset, batch_size=INFERENCE_BS, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    """
        Init proba result.
    """
    model_proba_dict = {}
    for _a, model in enumerate(models):
        model_proba_dict[str(_a)] = []
    
    # --------------------------------------------------- #
    # Prediction looop
    # * on dataset
    # --------------------------------------------------- #
    prediction_dict = {}
    for data in tqdm(loader):
        image = data['image'].to(device)

        # --------------------------------------------------- #
        # Ph.Prediction loop all models.
        # * {<Model>:[<Proba>[<Proba>]]}
        # --------------------------------------------------- #
        with torch.no_grad():
            probas = []
            probas_long = []
            for _i, model in enumerate(models):
                print("# --------------------------------------------------------- #")
                print("[INFO]Prediction Model:",_i)
                print("# --------------------------------------------------------- #")
                with torch.cuda.amp.autocast():
                    output = model(image)
                probas.append(output['clipwise_output'].detach().cpu().numpy().reshape(-1))
                model_proba_dict[str(_i)] = model_proba_dict[str(_i)]+output['clipwise_output'].detach().cpu().numpy().tolist()

    # --------------------------------------------------- #
    # Ph.Blending looop
    # --------------------------------------------------- #
    for index, row in test_df.iterrows():
        row_id = row['row_id']
        probas = []
        
        """
            bird weight
        """
        for k in model_proba_dict.keys():
            probas.append([x * y for (x, y) in zip(model_proba_dict[k][index], BIRD_WEIGHT)])
        
        """
            thresholding
        """
        probas = np.array(probas)
        if threshold_long is None:
            events = probas.mean(0) >= threshold
        else:
            events = ((probas.mean(0) >= threshold).astype(int) \
                      + (probas_long.mean(0) >= threshold_long).astype(int)) >= 2
        
        """
            convert labels
        """
        labels = np.argwhere(events).reshape(-1).tolist()
        if len(labels) == 0:
            prediction_dict[str(row_id)] = "nocall"
        else:
            labels_str_list = list(map(lambda x: CFG.target_columns[x], labels))
            label_string = " ".join(labels_str_list)
            prediction_dict[str(row_id)] = label_string
    return prediction_dict

In [ ]:
def prediction(test_audios,
               threshold=0.05, 
               threshold_long=None):
    
    # models = [model]
    warnings.filterwarnings("ignore")
    prediction_dicts = {}
    for audio_path in test_audios:
        with timer(f"Loading {str(audio_path)}", logger):
            clip, _ = sf.read(audio_path, always_2d=True)
            clip = np.mean(clip, 1)
            
        seconds = []
        row_ids = []
        for second in range(5, 65, 5):
            row_id = "_".join(audio_path.name.split(".")[:-1]) + f"_{second}"
            seconds.append(second)
            row_ids.append(row_id)
        print(row_ids)
        test_df = pd.DataFrame({
            "row_id": row_ids,
            "seconds": seconds
        })
        with timer(f"Prediction on {audio_path}", logger):
            prediction_dict = prediction_for_clip(test_df,
                                                  clip=clip,
                                                  models=models,
                                                  threshold=threshold, threshold_long=threshold_long)
        prediction_dicts.update(prediction_dict)
    return prediction_dicts

In [ ]:
threshold = 0.04
threshold_long = None # 0.05

prediction_dicts = prediction(test_audios=all_audios,
           threshold=threshold, 
           threshold_long=threshold_long)
# print(prediction_dicts)

for i in range(len(sample_submission)):
    sample = sample_submission.row_id[i]
    key = sample.split("_")[0] + "_" + sample.split("_")[1] + "_" + sample.split("_")[3]
    target_bird = sample.split("_")[2]
#     print(key, target_bird)
    if key in prediction_dicts:
        sample_submission.iat[i, 1] = (target_bird in prediction_dicts[key])
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
sample_submission

<h1 style="color: #6cb4e4;  text-align: center;  padding: 0.25em;  border-top: solid 2.5px #6cb4e4;  border-bottom: solid 2.5px #6cb4e4;  background: -webkit-repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);  background: repeating-linear-gradient(-45deg, #f0f8ff, #f0f8ff 3px,#e9f4ff 3px, #e9f4ff 7px);height:45px;">
<b>
EOF
</b></h1> 